In [19]:
import pandas as pd
import numpy as np
import glob
import csv
from datetime import datetime

In [20]:
# const
clinics = glob.glob('*.xlsx') #glob glob glob
clinics = [clinic[:-29] for clinic in clinics] # files have date ranges attached
# clinics = clinics[1:] # one clinic is outside this analysis
# used in loop to rename for next step in chain and easier reading
clinic_mapping = {
    'Ballard': 'Ballard',
    'BEAV': 'Beaverton',
    'Burlingame': 'Burlingame',
    'CherryCreek': 'Cherry Creek',
    'DC': 'D.C.',
    'DEN': 'Denver',
    'Kona': 'Kona',
    'LO': 'Lake Oswego',
    'MidCity': 'Mid City',
    'NOLA': 'Metairie',
    'NorthPortland': 'N. Portland',
    'Pearl': 'Pearl',
    'PORT': 'Portland',
    'SAR': 'Saratoga',
    'SFEC': 'SF - EC',
    'TANAS': 'Tanasbourne',
    'VANC': 'Vancouver',
    'ProgressRidge': 'Progress Ridge',
    'QueenAnne': 'Queen Anne',
    'PowellSquare': 'Powell Square',
    'HappyValley': 'Happy Valley',
    'Sunnyvale': 'Sunnyvale',
}

# marks when the analysis occurred
now = datetime.now()
month = now.strftime('%B').lower()
year = now.strftime('%Y')
filename = f'clinic_cpt_breakout_{month}_{year}.csv'

# there is a finite amount of correct columns and infinite incorrect values
columns = ['PT','All','97530','97535','97112','97110','97140','97001',
           '97162','97163','97161','97002','97164','98960','97116','97033',
           '97760','97150','97124','97039','97035','97032','97016','97014',
           '97012','97010','95853','95852','95851','95597','G0283','G0282',
           '20553','97608','92542','95992,29540','97610','64550','29240',
           '29260','29280','29520','29530','29550','29200','29799','20552',
           '97799','20053','97597','97750','97000','97774','97775','97776',
           '97777','97546','NC001']

In [21]:
def analyze_cpt(df):
    # pt eval codes
    eval_cols = ['97161', '97162', '97163']
    # re-evaluation codes
    reval = df[['97161', '97162', '97163', '97164']].sum()
    reval = reval.loc['97164'] / reval[eval_cols].sum()
    
    # # of eval codes verses # of codes
    total_eval = df.loc[(df[eval_cols] == 1.0).any(axis=1) & (df['All'] <= 10)] # <=10 removes total rows, since we are using margins = True later on
    total_eval = total_eval['All'].sum() / total_eval[eval_cols].sum().sum()
    
    # # of codes per treatment block when not a new diagnosis
    non_eval = df.loc[(df[eval_cols] == 0).all(axis=1)]
    non_eval = non_eval['All'].sum() / non_eval.shape[0]
    
    # high value billing code list
    code_list = ['97530', '97535', '97112', '97001', 
                '97162', '97163', '97161', '97002', 
                '97164', '92542', '64550', '29280', 
                '29520', '29530', '29550', '29200', 
                '20552', '97799', '20053', '97750']
    
    code_mix = df[code_list]
    code_mix = (code_mix.sum().sum() / df['All'].sum()) # % of codes in the high value blend

    try:
        # clinic, pt
        return writer.writerow([i,g, f'{code_mix:.0%}', f'{total_eval:.3}', f'{non_eval:.3}', f'{reval:.0%}'])
    except:
        # clinic, clinic
        return writer.writerow([i,i, f'{code_mix:.0%}', f'{total_eval:.3}', f'{non_eval:.3}', f'{reval:.0%}'])

In [22]:
with open(filename, 'w', newline='') as file_object:
    writer = csv.writer(file_object)
    writer.writerow(['Clinic', 'PT','Clinic Code Mix', 'Clinic Codes per Eval', 'Clinic Codes per Non-Eval', 'Clinic Re-eval per Eval'])
    
    for i in clinics:
        # import glob #glob is kinda funky sometimes
        glob = glob.glob(i+'*.xlsx')
        df = pd.concat(pd.read_excel(f) for f in glob)
        
        i = clinic_mapping[i] #changes name to actuals instead of external code

        # cleaning up columns with extraneous information
        df['CPT'] = df['CPT'].replace({'GP:':'', ':59':'', '.KX':''}, regex=True)
        df['PT'] = df['PT'].replace({r'[A-Z]{2,3} \(': '', r'[A-Z]{2} \(': '', '\)': ''}, regex=True)

        # the groupby version of this is a fair bit messier, and about the same speed
        pivot = np.round(pd.pivot_table(df, values= 'Units',
                                    index=['PT','Patient', 'Date of Service'],
                                    columns=['CPT'],
                                    aggfunc='sum',
                                    margins=True    
                                    ))

        # moving our pivot into a more manageable form
        clinic_df = pivot.reset_index().reindex(columns = columns).replace(np.nan, 0)
        # this removes the total row while keeping the total column 
        clinic_df = clinic_df[clinic_df['PT'] != 'All']

        # list of pts
        pt = clinic_df.PT.unique()

        analyze_cpt(clinic_df)
        
        # loop through individual providers
        for g in pt:
            # make a df for each provider for individual analysis
            pt_df = clinic_df.loc[(clinic_df['PT'].str.contains(g,regex =False))]
            pt_df['PT'] = g
            analyze_cpt(pt_df)

C:\Users\david\AppData\Local\Temp\ipykernel_11408\4005814693.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_df['PT'] = g
C:\Users\david\AppData\Local\Temp\ipykernel_11408\4005814693.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_df['PT'] = g
C:\Users\david\AppData\Local\Temp\ipykernel_11408\4005814693.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio